### ２人線形クールノー競争

In [48]:
import numpy as np

# Random number generator seed for reproducibility
np.random.seed(42)

# Random number of firms
N = np.random.randint(2, 6)  # Randomly choose between 2 and 5 firms

# Random production capacities
C = np.random.randint(5, 20, size=N)

# Random marginal costs
c = np.random.randint(1, 10, size=N)

# Random price function parameters
a = np.random.uniform(50, 150)
b = np.random.uniform(0.5, 5)

# Utility function for each firm
def utility(x, c, a, b):
    x_tot = np.sum(x)
    price = a - b * x_tot
    return x * price - c * x

# Gradient of the utility function (payoff gradient)
def payoff_gradient(x, i, a, b):
    x_tot = np.sum(x)
    price = a - b * x_tot
    return price - b * x[i] - c[i]

# Function to check diagonal strict concavity (monotonicity)
def is_monotone(x, lambdas, a, b):
    N = len(x)
    for i in range(N):
        for j in range(N):
            if i != j:
                grad_i = payoff_gradient(x, i, a, b)
                grad_j = payoff_gradient(x, j, a, b)
                diff_grad = grad_i - grad_j
                diff_x = x[i] - x[j]
                if not (np.dot(lambdas, diff_grad * diff_x) < 0).all():
                    return False
    return True

# Initial strategy profile
x = np.random.uniform(0, 10, size=N)

# Weights for DSC condition
lambdas = np.ones(N)

# Check if the game is monotone
monotone = is_monotone(x, lambdas, a, b)
print("The Cournot game is monotone:", monotone)

# Display the random parameters
print(f"Number of firms (N): {N}")
print(f"Production capacities (C): {C}")
print(f"Marginal costs (c): {c}")
print(f"Price function parameters (a, b): ({a}, {b})")
print(f"Initial strategy profile (x): {x}")


The Cournot game is monotone: True
Number of firms (N): 4
Production capacities (C): [ 8 17 19 15]
Marginal costs (c): [8 5 7 3]
Price function parameters (a, b): (59.99749158180029, 2.566620013846402)
Initial strategy profile (x): [3.33708611 1.42866818 6.50888473 0.56411579]


### どんなutilityがあるか確認

In [49]:
import sympy as sp

# Define symbolic variables
#N = sp.symbols('N')
x = sp.symbols('x1:%d' % (N+1))  # Quantity supplied by each firm
x_tot = sp.symbols('x_tot')  # Total quantity supplied
c = sp.symbols('c1:%d' % (N+1))  # Marginal costs
a, b = sp.symbols('a b')  # Price function parameters
d = sp.symbols('d1:%d' % (N+1))  # Quadratic cost coefficients
rho = sp.symbols('rho')  # Elasticity parameter
A, alpha, beta = sp.symbols('A alpha beta')  # Cobb-Douglas parameters

# Define total quantity supplied
x_tot_expr = sum(x)

# Define price function
price = a - b * x_tot

# 1. Linear cost function
utility_linear_cost = [x[i-1] * price - c[i-1] * x[i-1] for i in range(1, N+1)]

# 2. Quadratic cost function
utility_quadratic_cost = [x[i-1] * price - (c[i-1] * x[i-1] + 0.5 * d[i-1] * x[i-1]**2) for i in range(1, N+1)]

# 3. CES function
ces_price = (sum([x[i]**rho for i in range(N)])**(1/rho))
utility_ces = [x[i] * ces_price - c[i] * x[i] for i in range(N)]

# 4. Cobb-Douglas production function
cobb_douglas_production = [A * x[i]**alpha * (sum([x[j]**beta for j in range(N)]))**(1-alpha) - c[i] * x[i] for i in range(N)]

# Display the utility functions
print("Utility functions with linear cost:")
for i in range(N):
    print(f"u{i+1}: {utility_linear_cost[i]}")

print("\nUtility functions with quadratic cost:")
for i in range(N):
    print(f"u{i+1}: {utility_quadratic_cost[i]}")

print("\nUtility functions with CES function:")
for i in range(N):
    print(f"u{i+1}: {utility_ces[i]}")

print("\nUtility functions with Cobb-Douglas production function:")
for i in range(N):
    print(f"u{i+1}: {cobb_douglas_production[i]}")


Utility functions with linear cost:
u1: -c1*x1 + x1*(a - b*x_tot)
u2: -c2*x2 + x2*(a - b*x_tot)
u3: -c3*x3 + x3*(a - b*x_tot)
u4: -c4*x4 + x4*(a - b*x_tot)

Utility functions with quadratic cost:
u1: -c1*x1 - 0.5*d1*x1**2 + x1*(a - b*x_tot)
u2: -c2*x2 - 0.5*d2*x2**2 + x2*(a - b*x_tot)
u3: -c3*x3 - 0.5*d3*x3**2 + x3*(a - b*x_tot)
u4: -c4*x4 - 0.5*d4*x4**2 + x4*(a - b*x_tot)

Utility functions with CES function:
u1: -c1*x1 + x1*(x1**rho + x2**rho + x3**rho + x4**rho)**(1/rho)
u2: -c2*x2 + x2*(x1**rho + x2**rho + x3**rho + x4**rho)**(1/rho)
u3: -c3*x3 + x3*(x1**rho + x2**rho + x3**rho + x4**rho)**(1/rho)
u4: -c4*x4 + x4*(x1**rho + x2**rho + x3**rho + x4**rho)**(1/rho)

Utility functions with Cobb-Douglas production function:
u1: A*x1**alpha*(x1**beta + x2**beta + x3**beta + x4**beta)**(1 - alpha) - c1*x1
u2: A*x2**alpha*(x1**beta + x2**beta + x3**beta + x4**beta)**(1 - alpha) - c2*x2
u3: A*x3**alpha*(x1**beta + x2**beta + x3**beta + x4**beta)**(1 - alpha) - c3*x3
u4: A*x4**alpha*(x1**beta

### 2-player

In [50]:
import sympy as sp
import random

# Define the number of firms
N = 2  # We are using two firms

# Define symbolic variables for quantities supplied by each firm
x1, x2 = sp.symbols('x1 x2')
x = [x1, x2]

# Define symbolic variables for marginal costs
c1, c2 = sp.symbols('c1 c2')
c = [c1, c2]

# Define price function parameters
a, b = sp.symbols('a b')

# Define quadratic cost coefficient for the second firm
d2 = sp.symbols('d2')

# Define total quantity supplied
x_tot = x1 + x2

# Define price function
price = a - b * x_tot

# Utility functions
utility_1 = x1 * price - c1 * x1  # Linear cost for firm 1
utility_2 = x2 * price - (c2 * x2 + 0.5 * d2 * x2**2)  # Quadratic cost for firm 2

# Gradient (first derivatives) of the utility functions
grad_u1_x1 = sp.diff(utility_1, x1)
grad_u2_x2 = sp.diff(utility_2, x2)

# Gradients with respect to each firm's quantity
gradients = [grad_u1_x1, grad_u2_x2]

# Display the utility functions and their gradients
print("Utility function for firm 1:")
print(f"u1: {utility_1}")
print("Gradient of utility function for firm 1 with respect to x1:")
print(f"∇u1: {grad_u1_x1}")

print("\nUtility function for firm 2:")
print(f"u2: {utility_2}")
print("Gradient of utility function for firm 2 with respect to x2:")
print(f"∇u2: {grad_u2_x2}")

# Function to check diagonal strict concavity (monotonicity)
def is_monotone(gradients, lambdas):
    N = len(gradients)
    conditions = []
    for i in range(N):
        for j in range(N):
            if i != j:
                diff_grad = gradients[i] - gradients[j]
                condition = sp.simplify(lambdas[i] * diff_grad * (x[i] - x[j])) < 0
                conditions.append(condition)
    return conditions

# Weights for DSC condition
lambdas = sp.symbols('lambda1 lambda2')
lambda_vals = [lambdas[0], lambdas[1]]

# Check if the game is monotone
conditions = is_monotone(gradients, lambda_vals)
print("\nConditions for monotonicity:")
for cond in conditions:
    print(cond)

# Generate random numerical values for substitution
subs = {
    a: random.uniform(50, 150),
    b: random.uniform(0.5, 5),
    c1: random.uniform(1, 10),
    c2: random.uniform(1, 10),
    d2: random.uniform(0.1, 1),
    x1: random.uniform(0, 10),
    x2: random.uniform(0, 10),
    lambdas[0]: 1,
    lambdas[1]: 1
}

# Evaluate the conditions with substituted values
evaluated_conditions = [cond.subs(subs) for cond in conditions]
monotone_val = all(evaluated_conditions)

print("\nEvaluated conditions:")
for cond in evaluated_conditions:
    print(cond)

print("\nThe Cournot game with different utility functions is monotone:", monotone_val)

# Display the random parameters used for substitution
print("\nRandom parameters used for substitution:")
for key, value in subs.items():
    print(f"{key}: {value}")


Utility function for firm 1:
u1: -c1*x1 + x1*(a - b*(x1 + x2))
Gradient of utility function for firm 1 with respect to x1:
∇u1: a - b*x1 - b*(x1 + x2) - c1

Utility function for firm 2:
u2: -c2*x2 - 0.5*d2*x2**2 + x2*(a - b*(x1 + x2))
Gradient of utility function for firm 2 with respect to x2:
∇u2: a - b*x2 - b*(x1 + x2) - c2 - 1.0*d2*x2

Conditions for monotonicity:
lambda1*(x1 - x2)*(-b*x1 + b*x2 - c1 + c2 + 1.0*d2*x2) < 0
lambda2*(x1 - x2)*(-b*x1 + b*x2 - c1 + c2 + 1.0*d2*x2) < 0

Evaluated conditions:
True
True

The Cournot game with different utility functions is monotone: True

Random parameters used for substitution:
a: 126.0333627460707
b: 0.7099978132261449
c1: 4.507946852507314
c2: 1.8261319363955293
d2: 0.9121567736456567
x1: 4.172013364203324
x2: 8.640170655175801
lambda1: 1
lambda2: 1


### Multi Player

In [51]:
import sympy as sp
import random

# Define the number of firms randomly
#N = random.randint(2, 5)  # Randomly choose between 2 and 5 firms
N=4

# Define symbolic variables for quantities supplied by each firm
x = sp.symbols('x1:%d' % (N+1))

# Define symbolic variables for marginal costs
c = sp.symbols('c1:%d' % (N+1))

# Define price function parameters
a, b = sp.symbols('a b')

# Define quadratic cost coefficients
d = sp.symbols('d1:%d' % (N+1))

# Define CES elasticity parameter
rho = sp.symbols('rho')

# Define Cobb-Douglas parameters
A, alpha, beta = sp.symbols('A alpha beta')

# Define total quantity supplied
x_tot = sum(x)

# Define price function
price = a - b * x_tot

# Define utility functions
utilities = []
for i in range(N):
    choice = random.choice(['linear', 'quadratic', 'ces', 'cobb_douglas'])
    if choice == 'linear':
        utility = x[i] * price - c[i] * x[i]  # Linear cost for firm i
    elif choice == 'quadratic':
        utility = x[i] * price - (c[i] * x[i] + 0.5 * d[i] * x[i]**2)  # Quadratic cost for firm i
    elif choice == 'ces':
        ces_price = (sum([x[j]**rho for j in range(N)])**(1/rho))
        utility = x[i] * ces_price - c[i] * x[i]  # CES function for firm i
    elif choice == 'cobb_douglas':
        cobb_douglas_production = A * x[i]**alpha * (sum([x[j]**beta for j in range(N)]))**(1-alpha)
        utility = cobb_douglas_production - c[i] * x[i]  # Cobb-Douglas function for firm i
    utilities.append(utility)

# Gradient (first derivatives) of the utility functions
gradients = [sp.diff(utility, x[i]) for i, utility in enumerate(utilities)]

# Display the utility functions and their gradients
for i in range(N):
    print(f"Utility function for firm {i+1}:")
    print(f"u{i+1}: {utilities[i]}")
    print(f"Gradient of utility function for firm {i+1} with respect to x{i+1}:")
    print(f"∇u{i+1}: {gradients[i]}")
    print()

# Function to check diagonal strict concavity (monotonicity)
def is_monotone(gradients, lambdas):
    N = len(gradients)
    conditions = []
    for i in range(N):
        for j in range(N):
            if i != j:
                diff_grad = gradients[i] - gradients[j]
                condition = sp.simplify(lambdas[i] * diff_grad * (x[i] - x[j])) < 0
                conditions.append(condition)
    return conditions

# Weights for DSC condition
lambdas = sp.symbols('lambda1:%d' % (N+1))

# Check if the game is monotone
conditions = is_monotone(gradients, lambdas)
print("\nConditions for monotonicity:")
for cond in conditions:
    print(cond)

# Generate random numerical values for substitution
subs = {
    a: random.uniform(50, 150),
    b: random.uniform(0.5, 5),
    rho: random.uniform(0.5, 2),  # CES elasticity parameter
    A: random.uniform(0.5, 2),    # Cobb-Douglas parameter
    alpha: random.uniform(0.1, 0.9), 
    beta: random.uniform(0.1, 0.9),
}

for i in range(N):
    subs.update({
        c[i]: random.uniform(1, 10),
        d[i]: random.uniform(0.1, 1),
        x[i]: random.uniform(0, 10),
        lambdas[i]: 1,
    })

# Evaluate the conditions with substituted values
evaluated_conditions = [cond.subs(subs) for cond in conditions]
monotone_val = all(evaluated_conditions)

print("\nEvaluated conditions:")
for cond in evaluated_conditions:
    print(cond)

print("\nThe Cournot game with different utility functions is monotone:", monotone_val)

# Display the random parameters used for substitution
print("\nRandom parameters used for substitution:")
for key, value in subs.items():
    print(f"{key}: {value}")


Utility function for firm 1:
u1: A*x1**alpha*(x1**beta + x2**beta + x3**beta + x4**beta)**(1 - alpha) - c1*x1
Gradient of utility function for firm 1 with respect to x1:
∇u1: A*alpha*x1**alpha*(x1**beta + x2**beta + x3**beta + x4**beta)**(1 - alpha)/x1 + A*beta*x1**alpha*x1**beta*(1 - alpha)*(x1**beta + x2**beta + x3**beta + x4**beta)**(1 - alpha)/(x1*(x1**beta + x2**beta + x3**beta + x4**beta)) - c1

Utility function for firm 2:
u2: -c2*x2 + x2*(a - b*(x1 + x2 + x3 + x4))
Gradient of utility function for firm 2 with respect to x2:
∇u2: a - b*x2 - b*(x1 + x2 + x3 + x4) - c2

Utility function for firm 3:
u3: -c3*x3 + x3*(a - b*(x1 + x2 + x3 + x4))
Gradient of utility function for firm 3 with respect to x3:
∇u3: a - b*x3 - b*(x1 + x2 + x3 + x4) - c3

Utility function for firm 4:
u4: -c4*x4 + x4*(a - b*(x1 + x2 + x3 + x4))
Gradient of utility function for firm 4 with respect to x4:
∇u4: a - b*x4 - b*(x1 + x2 + x3 + x4) - c4


Conditions for monotonicity:
lambda1*(x1 - x2)*(A*alpha*x1**(al